In [1]:
'''Import libraries'''
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import wandb
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
import os
from torch.utils.data import Dataset
import torch
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score
from imblearn.under_sampling import RandomUnderSampler
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizerFast, BertForSequenceClassification, AdamW, Trainer, TrainingArguments
from tqdm import tqdm
from torch.nn import functional as F
import torch.nn as nn

wandb.login()

c:\Users\Ghiki\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: alberto-rodero557. Use `wandb login --relogin` to force relogin


True

In [3]:
'''Variables and parameters'''

SAMPLES_TO_TRAIN=-1
DIMENSIONS=200

N_LABELS=2
MAX_LEN = 256
EPOCHS=100
PATIENCE=10
LEARNING_RATE=.00005
WEIGHT_DECAY=.01
BATCH_SIZE=16
METRIC_FOR_BEST_MODEL='eval_loss'
if METRIC_FOR_BEST_MODEL=='eval_loss':
    GREATER_IS_BETTER = False
else:
    GREATER_IS_BETTER = True

In [4]:
'''Preparing dataset'''

import pickle

file_path = 'datasets/subtaskA_glove_train_dev_monolingual.pkl'

# Load the data from the pickle file
with open(file_path, 'rb') as file:
    data = pickle.load(file)

# Extract the individual datasets from the loaded data
train_x = data['train_x']
train_y = data['train_y']
dev_x = data['dev_x']
dev_y = data['dev_y']

if SAMPLES_TO_TRAIN!=-1:
    random_indices = np.random.choice(train_x.shape[0], size=SAMPLES_TO_TRAIN, replace=False)
    train_x = train_x[random_indices]
    train_y = train_y[random_indices]

    random_indices_dev = np.random.choice(dev_x.shape[0], size=int(train_x.shape[0] * 0.2), replace=False)
    dev_x = dev_x[random_indices_dev]
    dev_y = pd.Series(dev_y.to_numpy()[random_indices_dev])

print(train_x.shape)
print(train_y.shape)
print(dev_x.shape)
print(dev_y.shape)

print(type(train_y))
print(type(dev_y))

(119757, 200)
(119757,)
(5000, 200)
(5000,)
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


In [5]:
'''metrics'''

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    labels = p.label_ids
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    auc = roc_auc_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'auc': auc,
        'precision': precision,
        'recall': recall,
    }

In [6]:
class Data(Dataset):
    def __init__(self, X_train, y_train):
        self.X = torch.from_numpy(X_train.astype(np.float32))
        self.y = torch.from_numpy(y_train).type(torch.LongTensor)
        self.len = self.X.shape[0]

    def __getitem__(self, index):
        return {'input_ids': self.X[index], 'labels': self.y[index]}

    def __len__(self):
        return self.len

X_train, X_test, y_train, y_test = train_test_split(train_x, train_y.values, test_size=0.2, random_state=42)
traindata = Data(X_train, y_train)
testdata = Data(X_test, y_test)

devdata=Data(dev_x, dev_y.values)

In [7]:
# number of features (len of X cols)
input_dim = train_x.shape[-1]

# number of classes (unique of y)
output_dim = 2

class RNNModel(nn.Module):
    def __init__(self):
        super(RNNModel, self).__init__()
        
        self.lstm1 = nn.LSTM(input_dim, 512, batch_first=True)
        self.ln1 = nn.LayerNorm(512)
        self.dropout1 = nn.Dropout(0.2)
        
        self.lstm2 = nn.LSTM(512, 512, batch_first=True)
        self.ln2 = nn.LayerNorm(512)
        self.dropout2 = nn.Dropout(0.2)
        
        self.fc = nn.Linear(512, output_dim)
        
        self.loss = nn.CrossEntropyLoss()

    def forward(self, input_ids, labels=None):
        # print(f"Input shape: {input_ids.shape}")
        
        x, _ = self.lstm1(input_ids)
        # print(f"After LSTM1: {x.shape}")

        x = self.ln1(x)
        x = self.dropout1(x)
        
        # print(f"Before LSTM2: {x.shape}")
        
        x, _ = self.lstm2(x)
        # print(f"After LSTM2: {x.shape}")

        x = self.ln2(x)
        x = self.dropout2(x)
        
        x = self.fc(x)
        # print(f"Output shape: {x.shape}")
        
        outputs = (x,)
        if labels is not None:
            loss = self.loss(x, labels)
            outputs = (loss,) + outputs
            
        return (outputs if len(outputs) > 1 else outputs[0])

# Instantiate the model with appropriate dimensions
model = RNNModel()

In [8]:
from transformers import EarlyStoppingCallback

model = RNNModel()

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    warmup_steps=500,
    weight_decay=WEIGHT_DECAY,
    metric_for_best_model=METRIC_FOR_BEST_MODEL,
    greater_is_better=GREATER_IS_BETTER,
    logging_dir='./logs',
    logging_steps=15000,
    do_train=True,
    do_eval=True,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    push_to_hub=False,
    logging_first_step=False,
    load_best_model_at_end=True,
    save_total_limit=2,
    report_to="wandb"
)

# Create trainer
trainer = Trainer(
    model=model, 
    args=training_args, 
    train_dataset=traindata,
    eval_dataset=testdata, 
    compute_metrics=compute_metrics,# training dataset
    callbacks=[EarlyStoppingCallback(early_stopping_patience=PATIENCE)]
)

# Train the model
print(trainer.evaluate())

trainer.train()

print(trainer.evaluate())

 95%|█████████▍| 1417/1497 [00:01<00:00, 861.32it/s]

100%|██████████| 1497/1497 [00:02<00:00, 627.44it/s]


{'eval_loss': 0.8866190910339355, 'eval_accuracy': 0.47828991315965264, 'eval_f1': 0.6470854044283778, 'eval_auc': 0.5, 'eval_precision': 0.47828991315965264, 'eval_recall': 1.0, 'eval_runtime': 1.9408, 'eval_samples_per_second': 12341.101, 'eval_steps_per_second': 771.319}


                                                      
  1%|          | 6006/598800 [00:23<4:01:07, 40.97it/s]

{'eval_loss': 0.5327990651130676, 'eval_accuracy': 0.7395207080828323, 'eval_f1': 0.6770202412382876, 'eval_auc': 0.7324993718660363, 'eval_precision': 0.8318280117033456, 'eval_recall': 0.5707925977653632, 'eval_runtime': 1.9799, 'eval_samples_per_second': 12097.297, 'eval_steps_per_second': 756.081, 'epoch': 1.0}


...

{'loss': 0.2646, 'learning_rate': 3.1238509109142574e-05, 'epoch': 37.58}


                                                        
 38%|███▊      | 227544/598800 [14:38<23:52, 259.16it/s]


{'eval_loss': 0.31859129667282104, 'eval_accuracy': 0.8669839679358717, 'eval_f1': 0.8577424540096446, 'eval_auc': 0.8657955461948942, 'eval_precision': 0.8779707495429616, 'eval_recall': 0.838425279329609, 'eval_runtime': 1.9102, 'eval_samples_per_second': 12539.03, 'eval_steps_per_second': 783.689, 'epoch': 38.0}
{'train_runtime': 878.0052, 'train_samples_per_second': 10911.666, 'train_steps_per_second': 682.001, 'train_loss': 0.3412613080271529, 'epoch': 38.0}


100%|██████████| 1497/1497 [00:01<00:00, 792.03it/s]

{'eval_loss': 0.3158232867717743, 'eval_accuracy': 0.8643954575818303, 'eval_f1': 0.8555674137317681, 'eval_auc': 0.8633692390503509, 'eval_precision': 0.8720087019579406, 'eval_recall': 0.8397346368715084, 'eval_runtime': 1.8921, 'eval_samples_per_second': 12659.028, 'eval_steps_per_second': 791.189, 'epoch': 38.0}


In [9]:
trainer.evaluate(devdata)

100%|██████████| 313/313 [00:00<00:00, 705.31it/s]


{'eval_loss': 1.2049089670181274,
 'eval_accuracy': 0.551,
 'eval_f1': 0.3631205673758865,
 'eval_auc': 0.5509999999999999,
 'eval_precision': 0.624390243902439,
 'eval_recall': 0.256,
 'eval_runtime': 0.4448,
 'eval_samples_per_second': 11241.57,
 'eval_steps_per_second': 703.722,
 'epoch': 38.0}

In [10]:
trainer.save_model('SavedModels/'+'lstm_'+str(round(SAMPLES_TO_TRAIN/1000))+'k')

In [11]:
# All samples trained with training dataset
print(''''eval_loss': 0.3158232867717743, 'eval_accuracy': 0.8643954575818303, 'eval_f1': 0.8555674137317681, 'eval_auc': 0.8633692390503509, 'eval_precision': 0.8720087019579406, 'eval_recall': 0.8397346368715084''')

'eval_loss': 0.3158232867717743, 'eval_accuracy': 0.8643954575818303, 'eval_f1': 0.8555674137317681, 'eval_auc': 0.8633692390503509, 'eval_precision': 0.8720087019579406, 'eval_recall': 0.8397346368715084
